In [3]:

from typing import Optional
import os
import warnings
warnings.filterwarnings('ignore')
import google.generativeai as genai
import csv
genai.configure(api_key="hide")
model = genai.GenerativeModel("gemini-2.0-flash")

In [4]:
pdf_path="/Users/vijay/Desktop/Onfinance/icici copy.pdf"
output_path_csv="/Users/vijay/Desktop/Onfinance/FUNDS_FACT_SHEETS_FINAL.csv"
with open(output_path_csv, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title','Page_num','Subtopic','Content'])


In [7]:
def validate_pdf(file_path: str) -> bool:
    if not os.path.exists(file_path):
        print(f"Error: File not found at path: {file_path}")
        return False
    if not file_path.lower().endswith('.pdf'):
        print("Error: File is not a PDF")
        return False
    return True
def extract_text_from_pdf(page: str,num:int,num_pages:int) -> Optional[str]:
                extracted_text = []
                total_chars = 0
                text = page.extract_text()   
                extracted_text.append(text)
                total_chars += len(text)
                print(f"Processed page {num + 1}/{num_pages}")
            
                final_text = '\n'.join(extracted_text)
                print(f"\nExtraction complete! Total characters: {len(final_text)}")
                return final_text
                

def process(pdf_files):
    """Process a chunk of text and return both input and output for verification"""
    Prompt=[f"{SYS_PROMPT}",*pdf_files]
    processed_text = model.generate_content(
    Prompt,
    generation_config={
        "temperature": 0,
        "top_p": 0.9})
    return processed_text.text
def get_title(text):
    prompt="You are given a chunk of text from the given chunk of text interpret a title for the given text.Just write the title and no other detail just one most appropriate title O/P: title"
    conversation = [
    {"role": "user", "parts": [{"text": f"{prompt}\n\n{text[0:1000000]}"}]},
    ]
    title = model.generate_content(
    conversation,
    generation_config={
        "temperature": 0,
        "top_p": 0.9,
        "max_output_tokens": 25,
    })
    return title.text


In [8]:

SYS_PROMPT = """
You are a world class text pre-processor, here is the raw data from a PDF, please parse and return it in a way that is crispy and usable to a chatbot for answering all the questions based on it.

The raw data is messed up with new lines, Latex math and you will see fluff that we can remove completely. Basically take away any details that you think might be useless as the data given to chatbot.

Remember, the the question could be on any topic whatsoever so the issues listed above are not exhaustive

Please be smart with what you remove and be creative ok?

Remember DO NOT START SUMMARIZING THIS, YOU ARE ONLY CLEANING UP THE TEXT AND RE-WRITING WHEN NEEDED

DON'T remove any detail .You are just rewtritting to make it easier to pass to the chatbot.Include all the information present in your answer.Keep tinies of the details.Include the entire portfolio and details about it and also all the details related to funds
DO NOT MISS OUT ANY SINGLE VALUE OR INFORMATION not even the % to NAV derivative.
PLEASE DO NOT ADD MARKDOWN FORMATTING, STOP ADDING SPECIAL CHARACTERS THAT MARKDOWN CAPATILISATION ETC LIKES
DO NOT USE \" in the middle of the string of any column only use it at the start and finish.
ALSO RETURN THE TITLE FOR THE GIVEN CONTEXT.
Here is the text:
"""

In [9]:
from pypdf import PdfReader, PdfWriter
from pydantic import BaseModel, Field
from typing import List
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",api_key="hide")
import base64

In [10]:
class subTopic(BaseModel):
    sub_topic:List[str]=Field(description="list of the subtopics you extracted from the content")
    cont:List[str]=Field(description="list of the data string you extracted from the content one data string per subtopic")

In [11]:
class Title(BaseModel):
    title:str=Field(description="Most appropriate title for the given content")
    processed_text:str=Field(description="The entire content of the page without missing any information")


In [12]:

titles=[]

output_dir = "pages"
os.makedirs(output_dir, exist_ok=True)


reader = PdfReader(pdf_path)
for i, page in enumerate(reader.pages):
    writer = PdfWriter()
    writer.add_page(page)

    output_path = f"{output_dir}/page_{i+1}.pdf"
    with open(output_path, "wb") as out_file:
        writer.write(out_file)


for file in sorted(os.listdir(output_dir)):
    page_num=int(file.replace("page_","").replace(".pdf",""))
    num_pages=len(os.listdir(output_dir))
    print(file)
    pdf_files=[]
    print(f"Processing PDF with {num_pages} pages...")
    with open(f"{output_dir}/{file}", "rb") as f:
            pdf_files.append({
                "mime_type": "application/pdf",
                "data": f.read()
            })
            print(f"\nExtracting text {page_num}")
            processed_text=""
            processed_text =process(pdf_files)
            title=get_title(processed_text)
            parser = JsonOutputParser(pydantic_object=subTopic)
            prompt = PromptTemplate(
            template='''You are an assistant that transforms mutual fund scheme entries and content is:{processed_text}.The main title of the page is:{title}.Get relevant subtopics youself from the content.Do not force subtopics if there are no main subtopics there.Do not make each company an subtopic.Keep them general portfolio,indicators and so on.Keep all the information from the given content.DO NOT MISS ANY INFORMATION PRESRMT IN THE CONTENT.GIVE EACH string AS SEPARATE STRING Witch start and end with \" .No characters should be outside of \".BE VERY CAREFUL THE NUMBER OF ENTERIES IN subtopit list should be equal to enteries in data lust.BE VERY CAREFUL WITH THE FORMATTING\n"
                    \n{format_instructions}\n''',
            input_variables=["processed_text","title"],
            partial_variables={"format_instructions": parser.get_format_instructions()})
            chain = prompt | llm | parser
            res = chain.invoke({"processed_text": processed_text,"title":title})
            subtopics=res['sub_topic']
            contents=res['cont']
            print(len(subtopics),len(contents))
            print(subtopics)
            print(contents)
            while(len(subtopics)!=len(contents)):
                parser = JsonOutputParser(pydantic_object=subTopic)
                prompt = PromptTemplate(
                template='''You are an assistant that transforms mutual fund scheme entries and content is:{processed_text}.The main title of the page is:{title}.Get relevant subtopics youself from the content.The subtopics should be main heading of tables and pointers.Keep them general portfolio,indicators and so on.Keep all the information from the given content.DO NOT MISS ANY INFORMATION PRESRMT IN THE CONTENT.GIVE EACH string AS SEPARATE STRING Witch start and end with \" .No characters should be outside of \".BE VERY CAREFUL THE NUMBER OF ENTERIES IN subtopit list should be equal to enteries in data lust.BE VERY CAREFUL WITH THE FORMATTING.Strictly keep the length of both the string the same\n"
                        \n{format_instructions}\n''',
                input_variables=["processed_text","title"],
                partial_variables={"format_instructions": parser.get_format_instructions()})
                chain = prompt | llm | parser
                res = chain.invoke({"processed_text": processed_text,"title":title})
                subtopics=res['sub_topic']
                contents=res['cont']
                print(len(subtopics),len(contents))
                print(subtopics)
                print(contents)
            ind=0
            while(ind<min(len(subtopics),len(contents))):
                with open(output_path_csv, "a", encoding="utf-8") as out_f:
                    writer = csv.writer(out_f, quoting=csv.QUOTE_ALL)
                    writer.writerow([title, page_num, subtopics[ind].replace('"', ''), contents[ind].replace('"', '')])
                ind+=1
    page_num+=1
        
            

page_1.pdf
Processing PDF with 8 pages...

Extracting text 1
41 41
['Returns as on March 31, 2025', 'Fund Managers', 'Inception/Allotment date', 'Monthly AAUM as on 31-Mar-25', 'Closing AUM as on 31-Mar-25', 'Application Amount', 'Min.Addl.Investment', 'NAV (As on 31-Mar-25)', 'Exit Load', 'Total Expense Ratio', 'Indicative Investment Horizon', 'Portfolio as on March 31, 2025 - Overview', 'Portfolio as on March 31, 2025 - Automobiles', 'Portfolio as on March 31, 2025 - Banks', 'Portfolio as on March 31, 2025 - Beverages', 'Portfolio as on March 31, 2025 - Capital Markets', 'Portfolio as on March 31, 2025 - Cement & Cement Products', 'Portfolio as on March 31, 2025 - Construction', 'Portfolio as on March 31, 2025 - Consumable Fuels', 'Portfolio as on March 31, 2025 - Consumer Durables', 'Portfolio as on March 31, 2025 - Diversified Fmcg', 'Portfolio as on March 31, 2025 - Industrial Products', 'Portfolio as on March 31, 2025 - Insurance', 'Portfolio as on March 31, 2025 - It - Software'

In [13]:
directory_path = "pages"
for file in sorted(os.listdir(directory_path)):
    os.remove(f"{directory_path}/{file}")
if os.path.isdir(directory_path):
    os.rmdir(directory_path)

In [14]:
import json
import pandas as pd
from langchain_google_genai import ChatGoogleGenerativeAI


df = pd.read_csv(output_path_csv, encoding="utf-8")
content_index: dict[tuple[str,str], list[str]] = {}
for _, row in df.iterrows():
    key = (row["Title"], row["Subtopic"])
    content_index.setdefault(key, []).append(row["Content"])
page_index: dict[tuple[str,str], list[str]] = {}
for _, row in df.iterrows():
    key = (row["Title"], row["Subtopic"])
    page_index.setdefault(key, []).append(row["Page_num"])

# flatten for prompting
all_topics = sorted({t for t, _ in content_index})
all_subtopics = {
    t: sorted({st for tp, st in content_index if tp == t})
    for t in all_topics
}

api_key = "hide"


llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=api_key,
    temperature=1,
)
ts_list = "\n".join(f"- {t}: {', '.join(all_subtopics[t])}" for t in all_topics)
# Prompt to pick which topics/subtopics to use
def pick_topics_and_subtopics(query: str) -> list[dict]:

    prompt = f"""
You are a classifier. Given the list of topics & subtopics and a user question, return the all the needed topics subtopics list to answer in this JSON format Atleast give one topic subtopic pair.:

[
  {{ "topic": "TopicA", "subtopics": ["Sub1","Sub2"] }},
  ...
]

Topics & Subtopics:
{ts_list}

User question:
\"\"\"{query}\"\"\"
"""
    resp = llm.invoke(prompt)
    return json.loads(resp.content.strip(r"`\\njso"))

# Prompt to answer using only provided blocks
def answer_with_blocks(query: str, blocks: str) -> str:
    prompt = f"""
You are a helpful assistant. Use ONLY the provided content blocks to answer the question below.

User question:
\"\"\"{query}\"\"\"

Content blocks:
{blocks}

"""
    return llm.invoke(prompt)

# gather content for the selected picks
def gather_content(picks: list[dict],page_nums: list[int]) -> str:
    parts = []
    for pick in picks:
        for sub in pick["subtopics"]:
            parts.extend(content_index.get((pick["topic"], sub), []))
            page_nums.extend(page_index.get((pick["topic"], sub), []))
    return "\n-----\n".join(parts)

def rewrite(query,ts_list):
  prompt = f"""
You are a helpful assistant. Rewrite the given query so that it is easier for an llm such as you to take out the relevant topics subtopics pirs from the given list of topics and subtopics.

User question:
\"\"\"{query}\"\"\"

Topics & Subtopics:
{ts_list}
just write the needed information and no ither thing

"""
  return llm.invoke(prompt).content

if __name__ == "__main__":
    print("Ask anything (type 'exit' to quit):")
    while True:
        q = input("> ").strip()
        page_nums=[]
        if q.lower() in ("exit", "quit"):
            break
        query=rewrite(q,ts_list)
        picks = pick_topics_and_subtopics(query)
        blocks = gather_content(picks,page_nums)
        answer = answer_with_blocks(q, blocks)
        print("\nAnswer:\n", answer.content)
        print("\n— Used Topics/Subtopics —")



                                                 Title  Page_num  \
0    ICICI Prudential Bluechip Fund Analysis as of ...         1   
1    ICICI Prudential Bluechip Fund Analysis as of ...         1   
2    ICICI Prudential Bluechip Fund Analysis as of ...         1   
3    ICICI Prudential Bluechip Fund Analysis as of ...         1   
4    ICICI Prudential Bluechip Fund Analysis as of ...         1   
..                                                 ...       ...   
124  ICICI Prudential Dividend Yield Equity Fund An...         8   
125  ICICI Prudential Dividend Yield Equity Fund An...         8   
126  ICICI Prudential Dividend Yield Equity Fund An...         8   
127  ICICI Prudential Dividend Yield Equity Fund An...         8   
128  ICICI Prudential Dividend Yield Equity Fund An...         8   

                           Subtopic  \
0      Returns as on March 31, 2025   
1                     Fund Managers   
2          Inception/Allotment date   
3      Monthly AAUM as on 3

KeyboardInterrupt: Interrupted by user